## Imports

In [55]:
import sys
import random
from pathlib import Path
from collections import Counter

import datasets

from utils import combine_strings_with_whitespace

In [56]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/fsx/home-augustas/elk')

In [57]:
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path[:2]

['/fsx/home-augustas/elk/elk/promptsource', '/fsx/home-augustas/elk']

In [58]:
from templates import DatasetTemplates

In [59]:
# Disable the logging of the datasets library
import datasets

datasets.logging.set_verbosity_error()

# Load the dataset

In [60]:
!ls -lah datasets | grep {VERSION}

drwxr-xr-x 8 augustas Domain Users  25K Jun 28 22:24 burns_datasets_VINC_ppo_training_raw_v2
-rw-r--r-- 1 augustas Domain Users 3.2M Jun 28 23:40 burns_datasets_VINC_ppo_training_v2.parquet
drwxr-xr-x 8 augustas Domain Users  33K Jun 28 22:24 burns_datasets_VINC_raw_v2
-rw-r--r-- 1 augustas Domain Users 2.3M Jun 28 22:03 burns_datasets_VINC_train_v2.parquet
-rw-r--r-- 1 augustas Domain Users 592K Jun 28 23:39 burns_datasets_VINC_validation_v2.parquet


In [61]:
VERSION = "v3"

# Set dataset_dict to test_dataset
dataset_dict = datasets.DatasetDict.load_from_disk(
    f"datasets/burns_datasets_VINC_ppo_training_raw_{VERSION}"
)
dataset_dict

DatasetDict({
    ag_news: Dataset({
        features: ['text', 'label'],
        num_rows: 1983
    })
    amazon_polarity: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 1983
    })
    dbpedia_14: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 1983
    })
    imdb: Dataset({
        features: ['text', 'label'],
        num_rows: 1983
    })
    super_glue/boolq: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 1983
    })
    super_glue/copa: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 86
    })
})

In [62]:
sum(dataset.num_rows for dataset in dataset_dict.values())

10001

In [63]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

ag_news: 4
amazon_polarity: 2
dbpedia_14: 14
imdb: 2
super_glue/boolq: 2
super_glue/copa: 2


## Load the templates

In [64]:
dataset_template_dict = {}

for dataset_path in dataset_dict.keys():
    dataset_templates = DatasetTemplates(dataset_path)

    dataset_templates.templates = {
        x.name: x for x in dataset_templates.templates.values() if x.get_answer_choices_list(dataset_dict[dataset_path][0]) is not None
    }

    dataset_template_dict[dataset_path] = dataset_templates

In [65]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15
amazon_polarity: 11
dbpedia_14: 11
imdb: 13
super_glue/boolq: 10
super_glue/copa: 9


In [66]:
# dataset_template_dict["ag_news"].templates

In [67]:
for dataset_name in dataset_template_dict:
    good_templates = {
        name: x for name, x in dataset_template_dict[dataset_name].templates.items() if x.metadata.choices_in_prompt
    }
    dataset_template_dict[dataset_name].templates = good_templates
    print(f"{dataset_name}: {len(good_templates)}")

ag_news: 12
amazon_polarity: 8
dbpedia_14: 11
imdb: 4
super_glue/boolq: 5
super_glue/copa: 9


In [68]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 12
amazon_polarity: 8
dbpedia_14: 11
imdb: 4
super_glue/boolq: 5
super_glue/copa: 9


In [69]:
%%time

# Reproducibility
random.seed(2023)

ALLOWED_KEYS = ["prompt", "best_response", "original_dataset", "template_name"]

new_dataset = []

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)

    for idx, entry in enumerate(dataset):
        new_entry = entry.copy()
        
        # In case we need to know which dataset the entry came from
        new_entry["original_dataset"] = dataset_name

        # Sample a random template
        template_name = random.choice(
            list(dataset_template_dict[dataset_name].templates.keys())
        )
        template = dataset_template_dict[dataset_name].templates[template_name]
        new_entry["template_name"] = template_name

        q, a = template.apply(new_entry)
        new_entry["prompt"] = combine_strings_with_whitespace(a, q)
        if not new_entry["prompt"][-1].isspace():
            new_entry["prompt"] += " "

        # We can now change the label to whether the sample is truthful or not
        new_entry["best_response"] = a.strip()

        # Remove all other keys
        new_entry = { k: v for k, v in new_entry.items() if k in ALLOWED_KEYS }

        # Append to the new dataset
        new_dataset.append(new_entry)

ag_news


amazon_polarity
dbpedia_14
imdb
super_glue/boolq
super_glue/copa
CPU times: user 19.1 s, sys: 43.1 ms, total: 19.1 s
Wall time: 19.2 s


In [70]:
my_dataset = datasets.Dataset.from_list(new_dataset)
my_dataset

Dataset({
    features: ['original_dataset', 'template_name', 'prompt', 'best_response'],
    num_rows: 10001
})

In [71]:
my_dataset[0]

{'original_dataset': 'ag_news',
 'template_name': 'burns_4',
 'prompt': 'Too few games could set back PSP launch - Sony exec Signs of a delay, or just managing expectations? Choice 1: Science and technology. Choice 2: Business. Between choice 1 and choice 2, the topic of this example is ',
 'best_response': 'Science and technology'}

In [72]:
Counter(my_dataset["original_dataset"])

Counter({'ag_news': 1983,
         'amazon_polarity': 1983,
         'dbpedia_14': 1983,
         'imdb': 1983,
         'super_glue/boolq': 1983,
         'super_glue/copa': 86})

In [75]:
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_ppo_training_{VERSION}.parquet")

In [76]:
!ls -lah datasets | grep {VERSION}

drwxr-xr-x 7 augustas Domain Users  25K Jun 28 23:30 burns_datasets_VINC_ppo_training_raw_v3
-rw-r--r-- 1 augustas Domain Users 3.6M Jun 28 23:50 burns_datasets_VINC_ppo_training_v3.parquet
drwxr-xr-x 7 augustas Domain Users  33K Jun 28 23:30 burns_datasets_VINC_raw_v3
-rw-r--r-- 1 augustas Domain Users 3.5M Jun 28 23:24 burns_datasets_VINC_train_v3.parquet
-rw-r--r-- 1 augustas Domain Users 590K Jun 28 23:49 burns_datasets_VINC_validation_v3.parquet
